# To-do
- [ ] get the price, iv, hv and compute black_scholes price for the options
- [ ] make xPrice with get_prec based on greater of price and bs_price
- [ ] make cp_short, with the similar logic. Keep only one for each symbol (Put with higher strike)
- [ ] make protp_long and protc_short for protective calls and puts with similar logic
- [ ] make orders for all these covers and protects
- [ ] make orders for oprhans

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util  # type: ignore

util.startLoop()

# Check account values

In [ ]:
import asyncio

from src.ibfuncs import get_cushion, get_ib

with get_ib('SNP') as ib:
    acc = asyncio.run(get_cushion(ib))
display(acc)

# Get Base Data

In [ ]:
# get portfolio, undsymbols and openorders

import numpy as np

from ibfuncs import df_chains, df_iv, get_ib, get_open_orders, quick_pf
from snp import get_snp_unds
from utils import how_many_days_old, pickle_me

unds_path = ROOT/'data'/'snp_unds.pkl'
chains_path = ROOT /'data'/'chains.pkl'

# age check
age_should_be_less_than = 0.2 # Note: unds_path is used to check the age for all files
recreate = False

if how_many_days_old(unds_path) is None or (how_many_days_old(unds_path) > age_should_be_less_than):
    recreate = True

if recreate:
    unds = get_snp_unds()
    pickle_me(unds, unds_path)

else:
    unds = pd.read_pickle(unds_path)

with get_ib('SNP') as ib:

    unds_iv=ib.run(df_iv(ib=ib, stocks=unds, msg='first run ivs'))

    no_price=unds_iv[unds_iv[['price', 'hv', 'iv']].isnull().any(axis=1)].symbol.to_list()
    second_unds_iv = ib.run(df_iv(ib=ib, stocks=no_price, sleep_time=10, msg='second run ivs'))

    pf_raw = quick_pf(ib)
    oo = get_open_orders(ib)

    # Set symbol as index for both dataframes
    cols = ['symbol', 'price', 'iv', 'hv']

    unds_iv = unds_iv[cols].set_index('symbol')
    second_unds_iv = second_unds_iv[cols].set_index('symbol')

    # Update unds_iv with non-null values from second_unds_iv 
    unds_iv.update(second_unds_iv)

    # unds_iv = unds_iv.set_index('symbol')[['hv', 'iv', 'price']]
    unds_iv.columns = ['und_' + col for col in unds_iv.columns]
    unds_iv = unds_iv.reset_index()

    # ... add und_price
    pf = pf_raw.merge(unds_iv, on='symbol', how='left')

    # Update und_price with mktPrice where und_price is NaN and mktPrice has a value
    pf.loc[pd.isna(pf['und_price']) & pd.notna(pf['mktPrice']), 'und_price'] = pf['mktPrice']

    # Merge the DataFrames on 'symbol'
    unds_iv = unds_iv.merge(pf[['symbol', 'und_price']], on='symbol', how='left')

    # Fill NaN values in 'und_price_x' with values from 'und_price_y'
    unds_iv['und_price'] = unds_iv['und_price_x'].fillna(unds_iv['und_price_y'])

    # Drop the unnecessary 'und_price_x' and 'und_price_y' columns
    unds_iv = unds_iv.drop(columns=['und_price_x', 'und_price_y'])

    # ...temp store the pf, oo
    pf_path = ROOT / 'data' / 'pf.pkl'
    oo_path = ROOT / 'data' / 'oo.pkl'

    if recreate:
        chains = asyncio.run(df_chains(ib, unds, msg='raw chains'))
    else:
        chains = pd.read_pickle(chains_path)

pickle_me(pf, pf_path)
pickle_me(oo, oo_path)
pickle_me(chains, chains_path)

# Classify positions
Positions are classified as follows:
- `cwp`: the perfect position that is protected and has a cover.
- `exposed`: stocks that need to be covered and protected.
- `uncovered`: stocks that need to be only covered by options.
- `unprotected`: stocks that need to be only protected by options.
- `orphaned`: options that have no underlying stocks positions.
- `covering`: options that are covering positions.
- `protecting`: options that are portecting positions.

In [4]:
# Sort by covered with protection pairs
right_order = {'C': 0, '0': 1, 'P': 2}

pf = pf.sort_values(
    by=['symbol', 'right'],
    key=lambda x: x.map(right_order) if x.name == 'right' else x
)

# Initialize strategy field with blank underscore
pf['strategy'] = 'tbd'

# Filter for options only
opt_pf = pf[pf.secType == 'OPT']

# Group by symbol and expiry to find matching calls and puts
straddled = (opt_pf.groupby(['symbol', 'expiry', 'strike'])
                      .filter(lambda x: (
                          # Must have exactly 2 rows (call and put)
                          len(x) == 2 and 
                          # Must have both C and P
                          set(x['right']) == {'C', 'P'} and
                          # Position signs must match
                          np.sign(x['position'].iloc[0]) == np.sign(x['position'].iloc[1])
                      )))

# Update strategy field for straddles
pf.loc[pf.index.isin(straddled.index), 'strategy'] = 'straddled'

# Filter for stocks and their associated options
cwp = (pf.groupby('symbol')
                      .filter(lambda x: (
                          # Must have exactly one STK row
                          (x.secType == 'STK').sum() == 1 and
                          # Must have 1 or 2 OPT rows
                          (x.secType == 'OPT').sum() in [1, 2]
                      )))

# Update strategy field for covered calls/puts
pf.loc[pf.index.isin(cwp[cwp.right == 'C'].index), 'strategy'] = 'covering'
pf.loc[pf.index.isin(cwp[cwp.right == 'P'].index), 'strategy'] = 'protecting'

# Update strategy field for stocks with both covering and protecting
stocks_with_both = pf[(pf.secType == 'STK') & 
                      pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                      pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_with_both.index, 'strategy'] = 'cwp'

# Update strategy field for stocks with covering but no protecting
stocks_covered_only = pf[(pf.secType == 'STK') &
                        pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                        ~pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_covered_only.index, 'strategy'] = 'unprotected'

# Update strategy field for stocks with protecting but no covering  
stocks_protected_only = pf[(pf.secType == 'STK') &
                          ~pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                          pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_protected_only.index, 'strategy'] = 'uncovered'


# Update strategy field for orphaned options
pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'OPT'), 'strategy'] = 'orphaned'

# Update strategy field for exposed stock positions
pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'STK'), 'strategy'] = 'exposed'

pickle_me(pf, pf_path)

In [ ]:
# Check for null values in price, und_hv, and und_iv columns
null_rows = pf[pf[['und_price', 'und_hv', 'und_iv']].isnull().any(axis=1)]
print("Portfolio with null values in price, historical vol or implied vol:")
display(null_rows)

# Cook orders 
<b>For existing positions</b>
- `Exposed` and `Uncovered` stocks should be covered
   - ...both for long (covered call) and short (covered put)
- `Exposed` and `Unprotected` stocks should be protected
   - ...both for long (protective put) and short (protective call)
- `Orphaned` stocks should be liquidated

<b>For rest of the symbols</b>
- Symbols with announcements in a week need to be straddled
- Remaining ones should have naked puts

In [6]:
from utils import load_config

config = load_config('SNP')

# Get exposed and uncovered long
uncov = pf.strategy.isin(['exposed', 'uncovered'])
uncov_long = pf[uncov & (pf.position > 0)].reset_index(drop=True)

# Ready the chains for portfolio symbols
df_cc = (chains[chains.symbol.isin(uncov_long.symbol.unique())]
            .loc[(chains.dte.between(4, 11))]
            [['symbol', 'expiry', 'strike', 'dte']]
            .sort_values(['symbol', 'dte'])
            .reset_index(drop=True))

# Merge chains with underlying prices and volatilities
df_cc = df_cc.merge(unds_iv, on='symbol', how='left')

# Calculate standard deviation based on implied volatility and days to expiration
df_cc['sdev'] = df_cc.und_price * df_cc.und_iv * (df_cc.dte / 365)**0.5

# For each symbol and expiry, get 3 strikes above und_price + sdev

cc_std = config.get('COVER_STD_MULT')
no_of_options = 2

cc_long = (
    df_cc.groupby(['symbol', 'expiry'])
    .apply(lambda x: x[x['strike'] > x['und_price'] + cc_std * x['sdev']]
                    .assign(diff=abs(x['strike'] - (x['und_price'] + cc_std * x['sdev'])))
                    .sort_values('diff')
                    .head(no_of_options), include_groups=False)
    .reset_index()
    .drop(columns=['level_2', 'diff'])
)


In [ ]:
# Make covered call options
from ib_async import Option

from ibfuncs import qualify_me
from utils import clean_ib_util_df, get_dte

cov_calls = [
    Option(s, e, k, 'C', "SMART")
    for s, e, k in zip(cc_long.symbol, cc_long.expiry, cc_long.strike)
    ]

with get_ib('SNP') as ib:
    asyncio.run(qualify_me(ib, cov_calls))



df_cc1 = clean_ib_util_df([c for c in cov_calls if c.conId > 0])

# Get the lower of the covered call
df_ccf = df_cc1.loc[df_cc1.groupby('symbol')['strike'].idxmin()]


In [ ]:
# Get the price and IV of the covered calls
with get_ib('SNP') as ib:
    df_ccf = ib.run(df_iv(ib=ib, 
                          stocks=df_ccf.contract.to_list(), 
                          msg='cc option ivs', sleep_time=10)).drop(columns='hv')



In [23]:
# Merge the DataFrames on the 'symbol' column
merged_df = df_ccf.merge(df_cc.groupby('symbol').head(1)[['symbol', 'und_price', 'und_iv']], on='symbol', how='left')

# Reorder the columns to place 'und_price' and 'und_iv' as the 6th and 7th columns
columns = list(df_ccf.columns)[:5] + ['und_price', 'und_iv'] + list(df_ccf.columns)[5:]
merged_df = merged_df[columns]

# Calculate qty by dividing the position by 100
uncov_long['qty'] = uncov_long['position'] / 100

# Merge df_ccf with uncov_long on the 'symbol' column
merged_df = merged_df.merge(uncov_long[['symbol', 'qty']], on='symbol', how='left')

# Add the 'action' column with the value 'S'
merged_df['action'] = 'S'

merged_df.insert(4, 'dte', merged_df.expiry.apply(get_dte))

# Get the black scholes price
from src.utils import us_repo_rate, black_scholes

# Parameters for the Black-Scholes model
r = 0.01  # risk-free rate (1%)

# Convert dte from days to years
merged_df['T'] = merged_df['dte'] / 365.0

r = us_repo_rate()

# Calculate option prices using the black_scholes function
bs_price = merged_df.apply(lambda row: black_scholes(
    S=row['und_price'],
    K=row['strike'],
    T=row['T'],
    r=r,
    sigma=row['und_iv'],  # Use 'und_iv' for implied volatility
    option_type=row['right']
), axis=1)

merged_df['bs_price'] = bs_price

In [ ]:
merged_df

## Handle short stock positions

In [15]:
# Get exposed and uncovered short
uncov_short = pf[uncov & (pf.position < 0)].reset_index(drop=True)

# Ready the chains for portfolio symbols
df_cp = (chains[chains.symbol.isin(uncov_short.symbol.unique())]
         .loc[(chains.dte.between(4, 11))]
         [['symbol', 'expiry', 'strike', 'dte']]
         .sort_values(['symbol', 'dte'])
         .reset_index(drop=True))

# Merge chains with underlying prices and volatilities
df_cp = df_cp.merge(unds_iv, on='symbol', how='left')

# Calculate standard deviation based on implied volatility and days to expiration
df_cp['sdev'] = df_cp.und_price * df_cp.und_iv * (df_cp.dte / 365)**0.5

# For each symbol and expiry, get 3 strikes below und_price - sdev
cp_short = (
    df_cp.groupby(['symbol', 'expiry'])
    .apply(lambda x: x[x['strike'] < x['und_price'] - cc_std * x['sdev']]
               .assign(diff=abs(x['strike'] - (x['und_price'] - cc_std * x['sdev'])))
               .sort_values('diff')
               .head(no_of_options), include_groups=False)
    .reset_index()
    .drop(columns=['level_2', 'diff'])
)